In [1]:
from TPMetrics import TPMetrics
import configparser
import os
import pandas as pd
import numpy as np
import re
from itertools import repeat
from scipy.stats import rankdata

In [2]:
self = TPMetrics(r'/home/rafael/CNIC/Metabolomics/TurboPutativeCode/testing/std_test_TPMetric')
self.getClasses()
self.getCorrelations()
#self.writeOutfile()

In [3]:
l = self.df.loc[:, self.finalCols]

In [4]:
l = self.df.loc[:, [self.n, self.tpc]]

In [5]:
l = self.df.loc[:, [self.n, self.tpc]]

In [3]:
df = self.df.loc[:, ['index', self.a, self.tpc]].dropna()
df[self.tpc] = df[self.tpc].str.split(' // ')
df = df.explode(self.tpc)
dfl = list(zip(*[j for i,j in df.to_dict('list').items()]))

dfl = [
    (index, adduct, lipid, self.L2An[lipid], self.L2A2i[lipid][adduct])
    for index, adduct, lipid in dfl if lipid in self.L and adduct in self.L2A[lipid]
]

dfl = pd.DataFrame(
    dfl, columns=['index', self.a, self.tpc, self.s3An, self.s3Ai]
)

maxScore = 12
minScore = 6

dfl[self.s3s] = maxScore
dfl.loc[dfl[self.s3An]>1, self.s3s] = -((maxScore-minScore)/(dfl.loc[dfl[self.s3An]>1, self.s3An]-1))*(dfl.loc[dfl[self.s3An]>1, self.s3Ai]-1) + maxScore


df = pd.merge(
    df.loc[:, ['index', self.tpc]],
    dfl.loc[:, ['index', self.tpc, self.s3An, self.s3Ai, self.s3s]],
    on=['index', self.tpc],
    how='left'
)
df[self.s3s] = df[self.s3s].fillna(0)
df = df.groupby('index').agg(list).reset_index()

self.df = pd.merge(
    self.df,
    df.loc[:, ['index', self.s3An, self.s3Ai, self.s3s]],
    on='index',
    how='left'
)